# Asignación 2. Filtro Spam Bayes




In [8]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import GaussianNB
import math
from scipy.stats import *


In [2]:
#Agregar las columnas:
colNames = ['make','adress','all','3d','our','over','remove','internet','order','mail',
           'receive','will','people','report','addresses','free','busisnes','email','you',
           'credit','your','font','000','money','hp','hpl','george','650','lab','labs','telnet',
           '857','data','415','85','technology','1999','parts','pm','direct','cs','meeting',
           'original','project','re','edu','table','conference',';','(','[','!','$','#',
           'capital_run_length_average','capital_run_length_longest','capital_run_length_total','spam'
]

In [3]:
#Cargar los datos:
SPAMdf = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data",header=None, names=colNames) # Leer el archivo
#Vista previa de los datos:
SPAMdf.head()

,make,adress,all,3d,our,over,remove,internet,order,mail,...,;,(,[,!,$,#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


# Hacemos los cálculos del entrenamiento

In [4]:
entrenamiento, prueba = train_test_split(SPAMdf, test_size=0.2)#, random_state=18)

media_no_spam = entrenamiento[entrenamiento.spam==0].mean()
media_spam = entrenamiento[entrenamiento.spam==1].mean()

std_no_spam = entrenamiento[entrenamiento.spam==0].std()
std_spam = entrenamiento[entrenamiento.spam==1].std()

x_entrenamiento = entrenamiento.iloc[:,0:entrenamiento.shape[1]-1]
y_entrenamiento = entrenamiento.spam

x_prueba = prueba.iloc[:,0:prueba.shape[1]-1]
y_prueba = prueba.spam

pro_den_fun = norm(media_no_spam[0],std_no_spam[0])

# Definimos las funciones

In [5]:
def sets(data_frame):
    """
    Esta función divide los data frames en entrenamiento y prueba
    """
    entrenamiento, prueba = train_test_split(data_frame, test_size=0.2)#, random_state=18)
    return entrenamiento, prueba


def calcularProbabilidad(x, mean, std):
    """
    Esta función calcula la probabilidad de que la variable 'x' pertenezca a una pdf normal con media = mean y
    desviación estándar = std
    """
    exponente = math.exp(-(math.pow(x-mean,2)/(2*math.pow(std,2))))
    return (1 / (math.sqrt(2*math.pi) * std)) * exponente


def probabilidad_pertenencia_spam(n,df=x_entrenamiento):
    """
    Esta función sirve para obtener la probabilidad de que una observación pertenezca a la categoría de SPAM
    
    Función de: 
    n := Número de la observación
    df := El valor default es x_entrenamiento. Para hacer la inferencia hay que usar x_prueba
    """
    hola = np.zeros(df.shape[1])
    for i in range(0,len(hola)):
        if df.iloc[n,i] != 0:
            x = df.iloc[n,i]
            mean = media_spam[i]
            std = std_spam[i] 
            si = calcularProbabilidad(x, mean, std)
            hola[i] = si
    return(np.sum(hola)/len(hola))

def probabilidad_pertenencia_no_spam(n,df=x_entrenamiento):
    """
    Esta función sirve para obtener la probabilidad de que una observación no pertenezca a la categoría de SPAM
    
    Función de: 
    n := Número de la observación
    df := El valor default es x_entrenamiento. Para hacer la inferencia hay que usar x_prueba
    """
    hola = np.zeros(df.shape[1])
    for i in range(0,len(hola)):
        if df.iloc[n,i] != 0:
            x = df.iloc[n,i]
            mean = media_no_spam[i]
            std = std_no_spam[i] 
            si = calcularProbabilidad(x, mean, std)
            hola[i] = si
    return(np.sum(hola)/len(hola))

def es_o_no_es(df):
    es_o_no_es_prueba = np.zeros(df.shape[0])
    for i in range(0,df.shape[0]):
        if probabilidad_pertenencia_spam(i,df) > probabilidad_pertenencia_no_spam(i,df):
            es_o_no_es_prueba[i] = 1
    return es_o_no_es_prueba

def precision(df):
    es = es_o_no_es(df)
    probabilidad = (1- sum(abs(y_prueba - es))/len(es))*100
    #print('Pertenece a spam:',es)
    print('Pude predecir el spam el {0}% de las veces:',probabilidad)
    
##############Para Sklearn

def es_o_no_es_skl(df):
    clf = GaussianNB()
    clf.fit(x_entrenamiento, y_entrenamiento)
    return clf.predict(x_prueba)

def precision_skl(df):
    es = es_o_no_es_skl(df)
    probabilidad = (1- sum(abs(y_prueba - es))/len(es))*100
    #print('Pertenece a spam:',es)
    print('Pude predecir el spam el {0}% de las veces:',probabilidad)

# Hacemos la prueba Usando Probabilidades

In [6]:
precision(x_prueba)

Pude predecir el spam el {0}% de las veces: 67.5352877307


# Hacemos la prueba Usando Sklearn

In [7]:

precision_skl(x_prueba)

Pude predecir el spam el {0}% de las veces: 82.8447339848
